# Query CMIP6 ocean/atmos vert coords - native/CMIP6_CVs vs output

# Notes

## imports

In [1]:
%%time
import glob
import json
import os
os.sys.path.insert(0, "/p/user_pub/xclim/persist/software/xsearch/")
import xcdat as xc
import xsearch as xs

import logging
logging.getLogger("xcdat").setLevel(logging.ERROR)

CPU times: user 6.87 s, sys: 597 ms, total: 7.46 s
Wall time: 6 s


## read CMIP6_source_id.json

In [2]:
%%time
path = "/home/durack1/git/CMIP6_CVs"
file = "CMIP6_source_id.json"
with open(os.path.join(path, file), 'r') as f:
    js = json.load(f)
js.keys()
srcIds = js["source_id"]
srcIds.keys()

CPU times: user 2 ms, sys: 970 μs, total: 2.97 ms
Wall time: 2.37 ms


dict_keys(['4AOP-v1-5', 'ACCESS-CM2', 'ACCESS-ESM1-5', 'ACCESS-OM2', 'ACCESS-OM2-025', 'ARTS-2-3', 'AWI-CM-1-1-HR', 'AWI-CM-1-1-LR', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'AWI-ESM-1-REcoM', 'BCC-CSM2-HR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAM-MPAS-HR', 'CAM-MPAS-LR', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM1-1-CAM5-CMIP5', 'CESM1-CAM5-SE-HR', 'CESM1-CAM5-SE-LR', 'CESM1-WACCM-SC', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CIESM', 'CMCC-CM2-HR4', 'CMCC-CM2-SR5', 'CMCC-CM2-VHR4', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-1', 'CanESM5-CanOE', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'E3SM-2-0', 'E3SM-2-0-NARRM', 'E3SM-2-1', 'EC-Earth3', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-ESM-1', 'EC-Earth3-HR', 'EC-Earth3-LR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'EC-Earth3P', 'EC-Earth3P-HR', 'EC-Earth3P-VHR', 'ECMWF-IFS-HR', 'ECMWF-IFS-LR', 'ECMWF-IFS-MR', 'FGOALS-f3-H', 'FGOALS-f3-L', 'FGOALS-g3', 'FIO-ESM-2-0', 'GFDL-AM4', 'GFDL-CM4', 'GFDL-CM4C192', 'GF

# Ocean

## extract ocean level info

In [3]:
native = {}
for count, srcId in enumerate(srcIds):
    tmp = srcIds[srcId]["model_component"]["ocean"]["description"]
    tmp = tmp.split(";")
    for val in tmp:
        if "levels" in val:
            #print(srcId, val[:3])
            native[srcId] = int(val[:3])

## search for thetao

In [4]:
%%time
dpaths = xs.findPaths('historical', 'thetao', 'mon', realm='ocean', cmipTable='Omon', deduplicate=True)
#for count, sim in enumerate(dpaths):
#    print("{:03d}".format(count), sim)
#print(dpaths.keys())
#sims = xs.getValuesForFacet(dpaths, 'model', 'IPSL-CM5A2-INCA') ## 'E3SM-2-0')  # also E3SM-1-0 (E3SM-Project (1-5), UCSB, E3SM-Project), E3SM-1-1 + ECA, E3SM-2-0-NARRM
#sims

CPU times: user 52 ms, sys: 815 μs, total: 52.9 ms
Wall time: 101 ms


In [5]:
%%time
exp = "historical"
var = "thetao"
freq = "mon"
cmipTable = "Omon"
mipEra = "CMIP6"
firstMemberOnly = True

dpaths = xs.findPaths(exp, var, freq, cmipTable=cmipTable, mip_era=mipEra)
models = xs.natural_sort(xs.getGroupValues(dpaths, 'model'))
#print('models:', models)
#print('number of models:', len(models))

badFilesTime = [
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/historical/r1i1p1f1/Omon/thetao/gn/v20200624/thetao_Omon_EC-Earth3-AerChem_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-CC/historical/r1i1p1f1/Omon/thetao/gn/v20210113/thetao_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/historical/r1i1p1f1/Omon/thetao/gn/v20200919/thetao_Omon_EC-Earth3-Veg-LR_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/esgf_publish/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/historical/r1i1p1f1/Omon/thetao/gn/v20211207/thetao_Omon_EC-Earth3-Veg_historical_r1i1p1f1_gn_185001-185012.nc",
    "/p/css03/scratch/cmip6/CMIP/CAS/FGOALS-f3-L/historical/r1i1p1f1/Omon/thetao/gn/v20191007/thetao_Omon_FGOALS-f3-L_historical_r1i1p1f1_gn_185001-189912.nc",
];

aveLvls = []

for count, model in enumerate(models):
    dpaths_model = xs.retainDataByFacetValue(dpaths, 'model', model)
    members = xs.natural_sort(xs.getGroupValues(dpaths_model, 'member'))
    if firstMemberOnly:
        members = members[0:1]
    #print(count, model, members)
    for member in members:
        dpaths_model_member_list = xs.getValuesForFacet(dpaths_model, 'member', member)
        if len(dpaths_model_member_list) > 1:
            print('Error: multiple paths detected for ', model, member, ': ', dpaths_model_member_list)
        else:
            dpath = dpaths_model_member_list[0]
            ncfile = xs.natural_sort(glob.glob(os.path.join(dpath, '*.nc')))[0]
        # now poll vertical axis
        ds = xc.open_dataset(ncfile, decode_times=False)
        z = xc.get_dim_coords(ds, "Z")
        if "standard_name" in z.attrs.keys():
            sn = z.attrs["standard_name"]
        else:
            sn = "MISSING!"
        if "units" in z.attrs.keys():
            u = z.attrs["units"]
        else:
            u = "MISSING!"
        natZ = int(native[model])
        if natZ < len(z):
            s = "-- misreported --"
        elif len(z) != natZ:
            s = "-- decimated --"

        else:
            s = ""
        print(count, model, members, "standard_name/units:", sn, "/", u, "output:", len(z), "reported native:", natZ, s)
        aveLvls.append(len(z))

print(aveLvls)

0 ACCESS-CM2 ['r1i1p1f1'] standard_name/units: depth / m output: 50 reported native: 50 
1 ACCESS-ESM1-5 ['r1i1p1f1'] standard_name/units: depth / m output: 50 reported native: 50 
2 AWI-CM-1-1-MR ['r1i1p1f1'] standard_name/units: MISSING! / m output: 46 reported native: 46 
3 AWI-ESM-1-1-LR ['r1i1p1f1'] standard_name/units: MISSING! / m output: 46 reported native: 46 
4 BCC-CSM2-MR ['r1i1p1f1'] standard_name/units: depth / m output: 40 reported native: 40 
5 BCC-ESM1 ['r1i1p1f1'] standard_name/units: depth / m output: 40 reported native: 40 
6 CAMS-CSM1-0 ['r1i1p1f1'] standard_name/units: depth / m output: 50 reported native: 50 
7 CanESM5 ['r1i1p1f1'] standard_name/units: depth / m output: 45 reported native: 45 
8 CanESM5-1 ['r1i1p1f1'] standard_name/units: depth / m output: 45 reported native: 45 
9 CanESM5-CanOE ['r1i1p2f1'] standard_name/units: depth / m output: 45 reported native: 45 
10 CAS-ESM2-0 ['r1i1p1f1'] standard_name/units: depth / m output: 30 reported native: 30 
11 CE

/home/durack1/mambaforge/envs/xcd072nctax/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'thetao' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


13 CESM2-WACCM ['r1i1p1f1'] standard_name/units: olevel / m output: 33 reported native: 60 -- decimated --
14 CESM2-WACCM-FV2 ['r1i1p1f1'] standard_name/units: olevel / m output: 33 reported native: 60 -- decimated --
15 CIESM ['r1i1p1f1'] standard_name/units: depth / m output: 60 reported native: 46 -- misreported --
16 CMCC-CM2-HR4 ['r1i1p1f1'] standard_name/units: depth / m output: 50 reported native: 50 
17 CMCC-CM2-SR5 ['r1i1p1f1'] standard_name/units: depth / m output: 50 reported native: 50 
18 CMCC-ESM2 ['r1i1p1f1'] standard_name/units: depth / m output: 50 reported native: 50 
19 CNRM-CM6-1 ['r1i1p1f2'] standard_name/units: MISSING! / m output: 75 reported native: 75 
20 CNRM-CM6-1-HR ['r1i1p1f2'] standard_name/units: MISSING! / m output: 75 reported native: 75 
21 CNRM-ESM2-1 ['r1i1p1f2'] standard_name/units: MISSING! / m output: 75 reported native: 75 
22 E3SM-1-0 ['r1i1p1f1'] standard_name/units: depth / m output: 60 reported native: 60 
23 E3SM-1-1 ['r1i1p1f1'] standard_na

In [6]:
print("Average ocean levels:", (sum(aveLvls) // len(aveLvls) + 1))

Average ocean levels: 52


# Atmos

## extract atmos level info

In [7]:
native = {}
counter = 1
for count, srcId in enumerate(srcIds):
    tmp = srcIds[srcId]["model_component"]["atmos"]["description"]
    if tmp == "none":
        print(count, srcId, "none")
        continue
    tmp = tmp.split(";")
    for val in tmp:
        #print(srcId, val)
        if "levels" in val:
            #print(count, val)
            native[srcId] = int(val[:3])
        elif "72 vertical layers" in val:  # fudge E3SM-2-0, E3SM-2-1
            #print(count, srcId, val)
            native[srcId] = 72
        elif ". 72 vertical layers " in val:  # E3SM-2-0-NARRM
            #print(count, srcId, val)
            native[srcId] = 72

0 4AOP-v1-5 none
3 ACCESS-OM2 none
4 ACCESS-OM2-025 none
5 ARTS-2-3 none
66 GFDL-GRTCODE none
67 GFDL-OM4p5B none
68 GFDL-RFM-DISORT none
100 LBLRTM-12-8 none
122 RRTMG-LW-4-91 none
123 RRTMG-SW-4-02 none
124 RTE-RRTMGP-181204 none


## search for cl

In [8]:
%%time
exp = "historical"
var = "cl"
freq = "mon"
cmipTable = "Amon"
mipEra = "CMIP6"
firstMemberOnly = True

dpaths = xs.findPaths(exp, var, freq, cmipTable=cmipTable, mip_era=mipEra)
models = xs.natural_sort(xs.getGroupValues(dpaths, 'model'))
#print('models:', models)
#print('number of models:', len(models))

badFilesTime = [
];

aveLvls = []

for count, model in enumerate(models):
    #if model in ["E3SM-2-0", "E3SM-2-0-NARRM"]:
    #    continue # problem with variable
    dpaths_model = xs.retainDataByFacetValue(dpaths, 'model', model)
    members = xs.natural_sort(xs.getGroupValues(dpaths_model, 'member'))
    if firstMemberOnly:
        members = members[0:1]
    #print(count, model, members)
    for member in members:
        dpaths_model_member_list = xs.getValuesForFacet(dpaths_model, 'member', member)
        if len(dpaths_model_member_list) > 1:
            print('Error: multiple paths detected for ', model, member, ': ', dpaths_model_member_list)
        else:
            dpath = dpaths_model_member_list[0]
            ncfile = xs.natural_sort(glob.glob(os.path.join(dpath, '*.nc')))[0]
        # now poll vertical axis
        ds = xc.open_dataset(ncfile, decode_times=False)
        # deal with IPSL-CM6A-LR
        if model == "IPSL-CM6A-LR":
            z = ds["presnivs"]
        else:
            z = xc.get_dim_coords(ds, "Z")
        natZ = int(native[model])
        if natZ < len(z):
            s = "-- misreported --"
        elif len(z) != natZ:
            s = "-- decimated --"

        else:
            s = ""
        print(count, model, members, "output:", len(z), "reported native:", natZ, s)
        aveLvls.append(len(z))

print(aveLvls)

0 ACCESS-CM2 ['r1i1p1f1'] output: 85 reported native: 85 
1 ACCESS-ESM1-5 ['r1i1p1f1'] output: 38 reported native: 38 
2 AWI-ESM-1-1-LR ['r1i1p1f1'] output: 47 reported native: 47 
3 BCC-CSM2-MR ['r1i1p1f1'] output: 46 reported native: 46 
4 BCC-ESM1 ['r1i1p1f1'] output: 26 reported native: 26 
5 CAMS-CSM1-0 ['r1i1p1f1'] output: 31 reported native: 31 
6 CanESM5 ['r1i1p1f1'] output: 49 reported native: 49 
7 CanESM5-1 ['r1i1p1f1'] output: 49 reported native: 49 
8 CAS-ESM2-0 ['r1i1p1f1'] output: 35 reported native: 35 


/home/durack1/mambaforge/envs/xcd072nctax/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'cl' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


9 CESM2 ['r1i1p1f1'] output: 32 reported native: 32 
10 CESM2-FV2 ['r1i1p1f1'] output: 32 reported native: 32 
11 CESM2-WACCM ['r1i1p1f1'] output: 70 reported native: 70 
12 CESM2-WACCM-FV2 ['r1i1p1f1'] output: 70 reported native: 70 
13 CIESM ['r1i1p1f1'] output: 30 reported native: 30 
14 CMCC-CM2-HR4 ['r1i1p1f1'] output: 26 reported native: 26 
15 CMCC-CM2-SR5 ['r1i1p1f1'] output: 30 reported native: 30 
16 CMCC-ESM2 ['r1i1p1f1'] output: 30 reported native: 30 
17 CNRM-CM6-1 ['r1i1p1f2'] output: 91 reported native: 91 
18 CNRM-CM6-1-HR ['r1i1p1f2'] output: 91 reported native: 91 
19 CNRM-ESM2-1 ['r1i1p1f2'] output: 91 reported native: 91 
20 E3SM-1-0 ['r1i1p1f1'] output: 72 reported native: 72 
21 E3SM-1-1 ['r1i1p1f1'] output: 72 reported native: 72 
22 E3SM-1-1-ECA ['r1i1p1f1'] output: 72 reported native: 72 
23 E3SM-2-0 ['r1i1p1f1'] output: 72 reported native: 72 
24 E3SM-2-0-NARRM ['r1i1p1f1'] output: 72 reported native: 72 
25 EC-Earth3 ['r10i1p1f1'] output: 91 reported native: 

In [9]:
print("Average atmos levels:", (sum(aveLvls) // len(aveLvls) + 1))

Average atmos levels: 58
